In [22]:
# app.py
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, dash_table, Input, Output
from flask import Flask

# Flask server
server = Flask(__name__)
app = Dash(__name__, server=server)
app.title = "📈 Financial Dashboard"

# Load Data
df = pd.read_csv("financial_Accounting.csv")  # Place file in same folder
df["Date"] = pd.to_datetime(df["Date"])
df["Amount"] = df["Credit"].fillna(0) - df["Debit"].fillna(0)

# Layout
app.layout = html.Div(style={"backgroundColor": "#000", "padding": "20px"}, children=[
    html.H1("📈 Financial Dashboard", style={"textAlign": "center", "color": "#FFD700"}),

    html.Div([
        html.Div([
            html.Label("Select Category", style={"color": "white"}),
            dcc.Dropdown(
                options=[{"label": c, "value": c} for c in sorted(df["Category"].dropna().unique())],
                id="category-filter",
                placeholder="All Categories",
                style={"color": "#000"}
            ),
        ], style={"width": "30%", "display": "inline-block", "marginRight": "2%"}),

        html.Div([
            html.Label("Transaction Type", style={"color": "white"}),
            dcc.Dropdown(
                options=[{"label": t, "value": t} for t in sorted(df["Transaction_Type"].dropna().unique())],
                id="type-filter",
                placeholder="All Types",
                style={"color": "#000"}
            ),
        ], style={"width": "30%", "display": "inline-block", "marginRight": "2%"}),

        html.Div([
            html.Label("Select Date Range", style={"color": "white"}),
            dcc.DatePickerRange(
                id="date-filter",
                start_date=df["Date"].min(),
                end_date=df["Date"].max(),
                display_format="YYYY-MM-DD"
            )
        ], style={"width": "30%", "display": "inline-block"})
    ], style={"marginBottom": "30px"}),

    html.Div(id="kpi-cards", style={"display": "flex", "justifyContent": "space-around"}),

    dcc.Graph(id="monthly-bar-chart"),
    dcc.Graph(id="category-donut-chart"),

    html.Div([
        html.H3("🏆 Top 10 Transactions", style={"textAlign": "center", "color": "#FFD700"}),
        dash_table.DataTable(
            id="top10-table",
            style_table={"overflowX": "auto", "backgroundColor": "#111"},
            style_cell={"backgroundColor": "#111", "color": "#FFD700", "textAlign": "left"},
            style_header={"backgroundColor": "#222", "fontWeight": "bold", "color": "#FFD700"},
        )
    ])
])

# Callbacks
@app.callback(
    Output("kpi-cards", "children"),
    Output("monthly-bar-chart", "figure"),
    Output("category-donut-chart", "figure"),
    Output("top10-table", "data"),
    Output("top10-table", "columns"),
    Input("category-filter", "value"),
    Input("type-filter", "value"),
    Input("date-filter", "start_date"),
    Input("date-filter", "end_date")
)
def update_dashboard(category, tx_type, start_date, end_date):
    dff = df.copy()
    if category:
        dff = dff[dff["Category"] == category]
    if tx_type:
        dff = dff[dff["Transaction_Type"] == tx_type]
    if start_date and end_date:
        dff = dff[(dff["Date"] >= pd.to_datetime(start_date)) & (dff["Date"] <= pd.to_datetime(end_date))]

    income = dff[dff["Transaction_Type"] == "Income"]["Credit"].sum()
    expense = dff[dff["Transaction_Type"] == "Expense"]["Debit"].sum()
    balance = income - expense

    cards = [
        html.Div([
            html.Div("Total Income", style={"color": "#000"}),
            html.H3(f"${income:,.2f}", style={"color": "#000"})
        ], style={"backgroundColor": "#FFD700", "padding": "15px", "borderRadius": "10px", "width": "30%", "textAlign": "center"}),

        html.Div([
            html.Div("Total Expense", style={"color": "#000"}),
            html.H3(f"${expense:,.2f}", style={"color": "#000"})
        ], style={"backgroundColor": "#FFD700", "padding": "15px", "borderRadius": "10px", "width": "30%", "textAlign": "center"}),

        html.Div([
            html.Div("Balance", style={"color": "#000"}),
            html.H3(f"${balance:,.2f}", style={"color": "#000"})
        ], style={"backgroundColor": "#FFD700", "padding": "15px", "borderRadius": "10px", "width": "30%", "textAlign": "center"}),
    ]

    # Monthly Bar Chart
    monthly = dff.copy()
    monthly["Month"] = monthly["Date"].dt.strftime("%b")
    monthly_summary = monthly.groupby(["Month", "Transaction_Type"]).agg({"Credit": "sum", "Debit": "sum"}).reset_index()
    bar = px.bar(
        monthly_summary, x="Month", y=["Credit", "Debit"], color="Transaction_Type", barmode="group",
        title="Monthly Income vs Expense", template="plotly_dark",
        color_discrete_map={"Income": "green", "Expense": "red"}
    )

    # Donut Chart
    donut = px.pie(dff, names="Category", values="Debit", hole=0.4, template="plotly_dark",
                   title="Expense Distribution", color_discrete_sequence=px.colors.qualitative.Pastel)

    # Top 10 Transactions
    dff["Amount"] = dff["Credit"].fillna(0) - dff["Debit"].fillna(0)
    top10 = dff.sort_values("Amount", ascending=False).head(10)[["Date", "Description", "Category", "Transaction_Type", "Credit", "Debit", "Amount"]]
    table_data = top10.to_dict("records")
    table_columns = [{"name": col, "id": col} for col in top10.columns]

    return cards, bar, donut, table_data, table_columns

# Run app
if __name__ == "__main__":
    app.run(debug=True, port=8052)
